In [42]:
import pandas as pd
import numpy as np
import os
import preprocessor 
import re
import string
import spacy
from spacy.lang.en import stop_words as spacy_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [45]:
try:
    nlp = spacy.load('en_core_web_lg')
except:
    !spacy download en_core_web_lg
    nlp = spacy.load('en_core_web_lg')

# Load Data

In [4]:
folder_path = "../Data Twitter/Train/"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_train = pd.concat(dfs)
df_train.columns = ['text','label']
df_train['label_numeric'] = df_train['label'].astype('category').cat.codes
print('total train samples : ',len(df_train))
print(df_train['label'].value_counts())

class_mapping = list(df_train['label'].astype('category').cat.categories)

total train samples :  3613
fear       1147
anger       857
joy         823
sadness     786
Name: label, dtype: int64


In [5]:
folder_path = "../Data Twitter/Dev/"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_dev = pd.concat(dfs)
df_dev.columns = ['text','label']
df_dev['label_numeric'] = df_dev['label'].astype('category').cat.codes
print('total train samples : ',len(df_dev))
print(df_dev['label'].value_counts())

total train samples :  347
fear       110
anger       84
joy         79
sadness     74
Name: label, dtype: int64


# Preprocess Data

In [48]:
preprocessor.set_options(preprocessor.OPT.URL, preprocessor.OPT.MENTION)  # removes mentions and URLs only
stop_words = spacy_stopwords.STOP_WORDS
punctuations = string.punctuation

def preprocess(text):
    text = preprocessor.clean(text)
    text = re.sub(r'\W+', ' ', text)  # remove non-alphanumeric characters
    # replace numbers with the word 'number'
    text = re.sub(r"\d+", "number", text)
    text = text.lower()  # lower case everything
    
    return text.strip() # remove redundant spaces

In [49]:
df_train['text'] = df_train['text'].apply(preprocess)
df_dev['text'] = df_dev['text'].apply(preprocess)

# Prepare Embeddings

In [51]:
num_features = 300

X_train = np.zeros((len(df_train),num_features))
for i,text in enumerate(df_train['text']):
    for word in text.split(' '):
        X_train[i] += nlp(word).vector
    
X_test = np.zeros((len(df_dev),num_features))
for i,text in enumerate(df_dev['text']):
    for word in text.split(' '):
        X_test[i] += nlp(word).vector
    
Y_train = df_train['label_numeric']
Y_test = df_dev['label_numeric']

# Train Model

In [54]:
model = LogisticRegression(max_iter=200)
model.fit(X_train,Y_train)

/Users/ammarahmad/opt/anaconda3/envs/ai_project/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

# Predictions

In [55]:
Y_pred = model.predict(X_test)
print(classification_report(Y_test,Y_pred,target_names=class_mapping))

              precision    recall  f1-score   support

       anger       0.69      0.70      0.69        84
        fear       0.74      0.67      0.70       110
         joy       0.74      0.75      0.74        79
     sadness       0.67      0.73      0.70        74

    accuracy                           0.71       347
   macro avg       0.71      0.71      0.71       347
weighted avg       0.71      0.71      0.71       347

